In [8]:
# ! pip install pandas
! pip install sklearn

     |████████████████████████████████| 24.9 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 60.2 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=3fcfd069327b899e54cb10a24b2e2a87692aaac415629f003127d2c10d75099e
  Stored in directory: /root/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd

In [5]:
df = pd.read_pickle('withpixel_gender.pkl')

In [6]:
df

,full_path,gender,age,pixels
0,[17/10000217_1981-05-05_2009.jpg],1.0,28,"[255.0, 255.0, 255.0, 255.0, 255.0, 255.0, 255..."
2,[12/100012_1948-07-03_2008.jpg],1.0,60,"[92.0, 97.0, 91.0, 89.0, 94.0, 90.0, 91.0, 96...."
4,[16/10002116_1971-05-31_2012.jpg],0.0,41,"[61.0, 30.0, 10.0, 61.0, 30.0, 10.0, 61.0, 30...."
5,[02/10002702_1960-11-09_2012.jpg],0.0,52,"[97.0, 122.0, 178.0, 97.0, 122.0, 178.0, 97.0,..."
6,[41/10003541_1937-09-27_1971.jpg],1.0,34,"[190.0, 189.0, 194.0, 204.0, 203.0, 208.0, 203..."
...,...,...,...,...
62321,[38/9996938_1937-02-15_1968.jpg],1.0,31,"[71.0, 71.0, 71.0, 71.0, 71.0, 71.0, 71.0, 71...."
62322,[46/9996946_1943-11-01_1968.jpg],1.0,25,"[54.0, 54.0, 54.0, 44.0, 44.0, 44.0, 28.0, 28...."
62323,[49/9996949_1937-04-17_1963.jpg],1.0,26,"[41.0, 41.0, 41.0, 29.0, 29.0, 29.0, 22.0, 22...."
62325,[09/9998109_1972-12-27_2013.jpg],1.0,41,"[137.0, 174.0, 94.0, 137.0, 174.0, 94.0, 137.0..."


In [18]:
# since memory consumption is very high -we need to delete the df to get it back
# del df

In [9]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np

In [12]:
classes = 101 # 0 to 100
target = df['age'].values
target_classes = keras.utils.to_categorical(target, classes)

features = []

for i in range(0, df.shape[0]):
    features.append(df['pixels'].values[i])

features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

In [13]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.30)

# can delete features to free up ram
# delete df also


In [16]:
# del features # 20 gb ram freeedddd
# del df

In [10]:
# import pickle

# with open('image_features.pickle', 'wb') as handle:
#     pickle.dump(features, handle)
    
# target_gender = df['gender'].values

# with open('gender_target.pickle', 'wb') as handle:
#     pickle.dump(target_gender, handle)
    
# with open('age_target.pickle', 'wb') as handle:
#     pickle.dump(target, handle)

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation


# vgg-face model 

model = Sequential()
model.add(ZeroPadding2D((1,1), input_shape=(224,224,3)))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7,7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1,1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1,1)))
model.add(Flatten())
model.add(Activation('softmax'))



In [18]:
model.load_weights('vgg_face_weights.h5')

In [11]:
# ! git add .
# ! git commit -am "vgg defined"

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     7

In [20]:
from tensorflow.keras import Model

# lock the layer weights for early layers 
# - they could already detect some patterns
# fitting the network from scratch might cause to lose this info
# freeze all layers except last 3 conv layers - 2622 units
# just 101 units for age prediction task
# then add custom layer for 101 layers
for layer in model.layers[:-7]:
    layer.trainable = False

base_model_output = Sequential()
base_model_output = Convolution2D(101, (1,1), name='predictions')(model.layers[-4].output)
base_model_output = Flatten()(base_model_output)
base_model_output = Activation('softmax')(base_model_output)

age_model = Model(inputs=model.input, outputs=base_model_output)

In [21]:
age_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)     

In [ ]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

age_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='age_model.hdf5',
                              monitor='val_loss', verbose=1, save_best_only=True,
                              mode='auto')
scores=[]
epochs = 250; batch_size=256

for i in range(epochs):
    print('epoch',i)
    ix_train = np.random.choice(train_x.shape[0], size=batch_size)
    score = age_model.fit(train_x[ix_train], train_y[ix_train],
                         epochs=1, validation_data=(test_x, test_y),
                         callbacks=[checkpointer])
    scores.append(score)

epoch 0
8/8 [==============================] - ETA: 0s - loss: 14.9894 - accuracy: 0.0117
Epoch 00001: val_loss improved from inf to 5.48945, saving model to age_model.hdf5
8/8 [==============================] - 110s 14s/step - loss: 14.9894 - accuracy: 0.0117 - val_loss: 5.4894 - val_accuracy: 0.0193
epoch 1
8/8 [==============================] - ETA: 0s - loss: 7.1970 - accuracy: 0.0000e+00
Epoch 00001: val_loss improved from 5.48945 to 4.60942, saving model to age_model.hdf5
8/8 [==============================] - 97s 12s/step - loss: 7.1970 - accuracy: 0.0000e+00 - val_loss: 4.6094 - val_accuracy: 0.0264
epoch 2
8/8 [==============================] - ETA: 0s - loss: 5.4489 - accuracy: 0.0117
Epoch 00001: val_loss did not improve from 4.60942
8/8 [==============================] - 96s 12s/step - loss: 5.4489 - accuracy: 0.0117 - val_loss: 4.6127 - val_accuracy: 0.0239
epoch 3
8/8 [==============================] - ETA: 0s - loss: 5.9455 - accuracy: 0.0312
Epoch 00001: val_loss did no